In [32]:
import pandas as pd
import numpy as np
from pprint import pprint

In [53]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [54]:
# 3. Create a new set of 2,000 random latitudes and longitudes
coordinates = list(lat_lngs)

In [55]:
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

768

In [56]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [58]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [59]:
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        req = requests.get(city_url)
        city_weather =req.json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        city_desc = city_weather["weather"][0]["description"]
        # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                         "Weather Desc": city_desc})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jiuquan
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | turinsk
Processing Record 7 of Set 1 | cidreira
Processing Record 8 of Set 1 | manicore
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | bandarbeyla
Processing Record 11 of Set 1 | zeya
Processing Record 12 of Set 1 | carnarvon
Processing Record 13 of Set 1 | nanortalik
Processing Record 14 of Set 1 | la baule-escoublac
Processing Record 15 of Set 1 | plainview
Processing Record 16 of Set 1 | port alfred
Processing Record 17 of Set 1 | alta floresta
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | amderma
City not found. Skipping...
Processing Record 20 of Set 1 | saint-philippe
Processing Record 21 of Set 1 | east london
Processing Record 22 of Set 1 | su

In [60]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,Jiuquan,CN,39.7432,98.5174,61.39,26,0,17.90,clear sky
1,Hermanus,ZA,-34.4187,19.2345,56.89,86,44,2.71,scattered clouds
2,Punta Arenas,CL,-53.1500,-70.9167,50.11,71,75,13.80,light rain
3,Albany,US,42.6001,-73.9662,71.92,62,2,6.71,clear sky
4,Mataura,NZ,-46.1927,168.8643,49.48,93,52,9.69,broken clouds


In [61]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")